In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from resnet_features import *
from receptive_field import *
from ppnet import *

In [3]:
img_size = 224
prototype_shape = (200, 128, 1, 1)
num_classes = 20
prototype_activation_function = 'log'

In [4]:
ppnet = ProtoPNet.construct_PPNet(base_architecture='resnet50',
                              pretrained=True,
                              img_size=img_size,
                              prot_shape=prototype_shape,
                              num_classes=num_classes,
                              prototype_activation_function=prototype_activation_function,
                              add_on_layers_type='regular')

In [5]:
inp = torch.rand([10, 3, 224, 224])

In [6]:
out = ppnet(inp)

In [7]:
out[0].shape

torch.Size([10, 20])

In [8]:
out[1].shape

torch.Size([10, 200])

In [9]:
ppnet.prot_shape

(200, 128, 1, 1)

In [10]:
model_client = torch.nn.DataParallel(ppnet)

In [11]:
model_client.module.prot_class_id.shape

torch.Size([200, 20])

In [12]:
label = [1, 2, 6, 10, 2]

In [13]:
torch.t(model_client.module.prot_class_id[:, label]).shape

torch.Size([5, 200])

In [14]:
max_dist = (model_client.module.prot_shape[1] #module
                            * model_client.module.prot_shape[2]
                            * model_client.module.prot_shape[3]) 

In [15]:
max_dist

128

In [16]:
model_client.module.prot_class_id[0, ]

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.])

In [17]:
params = list(ppnet.named_parameters())

In [18]:
params = list(ppnet.features.named_parameters())

In [19]:
params[0]

('conv1.weight',
 Parameter containing:
 tensor([[[[ 1.3335e-02,  1.4664e-02, -1.5351e-02,  ..., -4.0896e-02,
            -4.3034e-02, -7.0755e-02],
           [ 4.1205e-03,  5.8477e-03,  1.4948e-02,  ...,  2.2060e-03,
            -2.0912e-02, -3.8517e-02],
           [ 2.2331e-02,  2.3595e-02,  1.6120e-02,  ...,  1.0281e-01,
             6.2641e-02,  5.1977e-02],
           ...,
           [-9.0349e-04,  2.7767e-02, -1.0105e-02,  ..., -1.2722e-01,
            -7.6604e-02,  7.8453e-03],
           [ 3.5894e-03,  4.8006e-02,  6.2051e-02,  ...,  2.4267e-02,
            -3.3662e-02, -1.5709e-02],
           [-8.0029e-02, -3.2238e-02, -1.7808e-02,  ...,  3.5359e-02,
             2.2439e-02,  1.7077e-03]],
 
          [[-1.8452e-02,  1.1415e-02,  2.3850e-02,  ...,  5.3736e-02,
             4.4022e-02, -9.4675e-03],
           [-7.7273e-03,  1.8890e-02,  6.7981e-02,  ...,  1.5956e-01,
             1.4606e-01,  1.1999e-01],
           [-4.6013e-02, -7.6075e-02, -8.9648e-02,  ...,  1.2108e-01,

In [20]:
ppnet.features

resnet50_features

In [21]:
from torchsummary import summary

In [23]:
ppnet

ProtoPNet(
  (features): resnet50_features
  (add_on_layers): Sequential(
    (0): Conv2d(2048, 128, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
    (3): Sigmoid()
  )
  (last_layer): Linear(in_features=200, out_features=20, bias=False)
)